# ロボットアーム - Panda Gym

In [ ]:
import os
import json
from pathlib import Path
import numpy as np
import gymnasium as gym
import panda_gym
import matplotlib.pyplot as plt
from matplotlib import font_manager
from tqdm import tqdm
from IPython.display import display, Markdown, Image

import torch

from gail_algo import GAILTrainer, build_expert_loader
from collect_expert_trajectories import collect_expert_trajectories

# 日本語フォント設定
import matplotlib
import japanize_matplotlib

* ロボット: Franka Emika Panda マニピュレータの操作タスクをシミュレート。
<br>

* 観測空間:
    - すべてのタスクにグリッパの位置と速度（6値）。
    - 物体を扱うタスクでは位置・姿勢・線形/回転速度が追加（物体1つにつき12値）。
    - グリッパ開閉距離（拘束されていなければ1値）。

* 行動空間:
    - グリッパの平行移動コマンド（x, y, z の3値）。
    - グリッパ開閉コマンド（1値）。
    
* シミュレーション:
    - エージェントの1ステップあたり20タイムステップ（各2ms）。
    - インタラクション周波数は25Hz。
    - ほとんどのタスクは約2秒（50ステップ）のエピソード長。
    
* 報酬関数:
    - 既定の報酬はスパース: 目標到達（5cm 以内）なら0、それ以外は -1。
    - スパース報酬は定義が簡単だが、進捗の手掛かりが少ない。

# PandaReach-v3

* ターゲット位置は30cm × 30cm × 30cmの範囲でランダム生成され、グリッパがそこへ到達するタスク。

In [ ]:
env = gym.make(
    "PandaReach-v3",
    render_mode="rgb_array",
    renderer="OpenGL",
    render_target_position=[0, 0.15, 0.25],
    render_distance=0.85,
    render_yaw=135,
    render_pitch=-20,
)

In [ ]:
print('\n観測空間:', env.observation_space)
print('\n行動空間: ', env.action_space)

obs, info = env.reset()
print('\n初期状態: ', (obs, info))

# 行動は環境のアクション空間と同じ形状にする（スカラーだとエラーになる）
action_sample = np.zeros(env.action_space.shape, dtype=np.float32)
print('\n環境での1ステップ: ', env.step(action_sample))

print('\n\nレンダリングした環境: ')
env.reset()
plt.axis('off')
plt.imshow(env.render())
plt.show()

In [ ]:
obs_shape = env.observation_space['observation'].shape[0] + \
            env.observation_space['achieved_goal'].shape[0] + \
            env.observation_space['desired_goal'].shape[0]

# GAIL用のエキスパート軌跡

1. `../TD3/Models/Expert/` の事前学習TD3エキスパートで軌跡（状態 + 行動）をロールアウト。
2. 生成した軌跡を一度 `./expert_trajectories.pt` に保存し、以降の実行で再利用。
3. このファイルからミニバッチを作り、学習中にGAIL識別器へ供給する。

In [ ]:
expert_path = "./expert_trajectories.pt"
if not os.path.exists(expert_path):
    collect_expert_trajectories(env_name="PandaReach-v3", episodes=200, steps_per_episode=300,
                                expert_model_path="../TD3/Models/Expert/", save_path=expert_path,
                                render=False)
else:
    print(f"キャッシュされたエキスパート軌跡を {expert_path} から利用します")

In [ ]:
expert_loader = build_expert_loader(expert_path, batch_size=256, device=None, shuffle=True)

# GAIL目的関数（数式）
- **報酬（ポリシー側）**: $r(s,a) = \log D_\phi(s,a)$。識別器が「エキスパートらしい」と判断するほど報酬が高い。
- **識別器の損失**（最小化で表記）: 
$$\mathcal{L}_D = -\mathbb{E}_{(s,a)\sim\text{expert}}[\log D_\phi(s,a)] - \mathbb{E}_{(s,a)\sim\pi_\theta}[\log (1 - D_\phi(s,a))]$$
- **ポリシー（生成側）の目的**: 
$$\max_\theta\; \mathbb{E}_{(s,a)\sim\pi_\theta}[\log D_\phi(s,a)]$$
- **TD3本体**: Actor/Critic はTD3そのまま（2つのCriticで $\min(Q_1, Q_2)$、ターゲット平滑化、遅延Actor更新）。GAILでは上記の $\log D$ を外部報酬として TD3 学習に渡す点のみが異なる。

In [ ]:
# GAILエージェント設定（繰り返し実行用に辞書で保持）
gail_agent_kwargs = dict(
    env=env,
    input_dims=obs_shape,
    agent_name='GAIL',
    model_save_path='./Models/Apprentice/',
    exploration_period=300,
    disc_lr=3e-4,
    disc_updates=2,
    gail_reward_scale=1.0,
    expert_loader=expert_loader,
)

print('GAILエージェント設定を初期化しました。次のセルで繰り返し学習できます。')

In [ ]:
# Auto-train GAIL agents for a fixed number of runs (no interactive prompts)

max_runs = 10  # number of sequential GAIL agents to train



run_idx = 0



results_dir = Path("../Results/GAIL")
results_dir.mkdir(parents=True, exist_ok=True)
manifest_path = results_dir / "run_manifest.json"



# Load existing manifest if it exists (resume-friendly)
if manifest_path.exists():
    try:
        with open(manifest_path, "r") as f:
            run_manifest = json.load(f)
    except Exception:
        run_manifest = []
else:
    run_manifest = []



# Track per-run aggregates for a final summary
per_run_mean_rewards = []  # mean episodic reward over evaluation rollouts
per_run_success_rates = []  # success rate (%) over evaluation rollouts



for run_idx in range(1, max_runs + 1):

    # Separate checkpoint directories per run to avoid overwriting
    gail_agent_kwargs['model_save_path'] = f"./Models/Apprentice_Run_{run_idx}/"

    agent = GAILTrainer(**gail_agent_kwargs)

    # Train the agent once; each run is independent
    score_history, avg_score_history, success_history, avg_success_history = agent.gail_train(
        n_episodes=500,
        opt_steps=10,
        print_every=50,
        render_save_path=None,
        plot_save_path=f"../Results/GAIL/GAIL_Performance_Run_{run_idx}.png",
        plot_title=f"GAIL Run {run_idx} Learning Curve",
    )

    agent.save_model()
    print(f"Run {run_idx} を保存しました -> {gail_agent_kwargs['model_save_path']}")

    # Post-training evaluation across multiple episodes to compute success rate
    eval_returns = []
    eval_successes = []
    for _ in range(10):
        ep_ret, success = agent.test_model(env=env, steps=200, render_save_path=None)
        eval_returns.append(ep_ret)
        eval_successes.append(1 if success else 0)

    eval_mean_r = float(np.mean(eval_returns))
    eval_succ_rate = float(np.mean(eval_successes) * 100)
    per_run_mean_rewards.append(eval_mean_r)
    per_run_success_rates.append(eval_succ_rate)
    print(f"Run {run_idx} 評価結果: 平均報酬 {eval_mean_r:.3f}, 成功率 {eval_succ_rate:.1f}%")

    # Persist learning history for this run
    history_path = results_dir / f"GAIL_History_Run_{run_idx}.json"
    with open(history_path, "w") as f:
        json.dump({
            "score_history": score_history,
            "avg_score_history": avg_score_history,
            "success_history": success_history,
            "avg_success_history": avg_success_history,
            "eval_mean_reward": eval_mean_r,
            "eval_success_rate": eval_succ_rate
        }, f)

    # Update manifest to keep track of all runs
    run_manifest.append({
        "run_idx": run_idx,
        "history_path": str(history_path),
        "plot_path": f"../Results/GAIL/GAIL_Performance_Run_{run_idx}.png",
        "model_path": gail_agent_kwargs['model_save_path'],
        "eval_mean_reward": eval_mean_r,
        "eval_success_rate": eval_succ_rate,
    })
    with open(manifest_path, "w") as f:
        json.dump(run_manifest, f, indent=2)

    # Display the most recent learning curve inline
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(score_history, label='Score')
    ax.plot(avg_score_history, label='Average Score')
    ax.set_title(f'GAIL Run {run_idx} Learning Curve')
    ax.set_xlabel('Episode')
    ax.set_ylabel('Score')
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    display(Markdown(f"### Run {run_idx} Learning Results"))
    display(fig)
    plt.close(fig)


# Summary over all automated runs
overall_mean_reward = float(np.mean(per_run_mean_rewards)) if per_run_mean_rewards else 0.0
overall_success_rate = float(np.mean(per_run_success_rates)) if per_run_success_rates else 0.0
print("\n===== GAIL Auto-Training Summary =====")
print(f"Runs executed: {len(per_run_mean_rewards)} / {max_runs}")
print(f"Average reward across runs: {overall_mean_reward:.3f}")
print(f"Average success rate across runs: {overall_success_rate:.1f}%")

# GAILポリシーの評価

In [ ]:
# 保存済みの各Runの学習曲線をまとめて表示
from glob import glob

results_dir = Path("../Results/GAIL")
manifest_path = results_dir / "run_manifest.json"

if not manifest_path.exists():
    print("まだ保存済みのRunがありません。学習セルを先に実行してください。")
else:
    with open(manifest_path, "r") as f:
        run_manifest = json.load(f)

    if len(run_manifest) == 0:
        print("マニフェストが空です。学習セルを先に実行してください。")
    else:
        for entry in run_manifest:
            run_idx = entry.get("run_idx")
            history_path = entry.get("history_path")
            if not history_path or not Path(history_path).exists():
                print(f"Run {run_idx}: 履歴ファイルが見つかりませんでした ({history_path})")
                continue

            with open(history_path, "r") as f:
                hist = json.load(f)
            score_history = hist.get("score_history", [])
            avg_score_history = hist.get("avg_score_history", [])
            success_history = hist.get("success_history", [])
            avg_success_history = hist.get("avg_success_history", [])

            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
            # 報酬
            axes[0].plot(score_history, label='Score')
            axes[0].plot(avg_score_history, label='Average Score')
            axes[0].set_title(f'GAIL Run {run_idx} Learning Curve')
            axes[0].set_xlabel('Episode')
            axes[0].set_ylabel('Score')
            axes[0].legend()
            axes[0].grid(True, linestyle='--', alpha=0.5)

            # 成功率（存在する場合のみプロット）
            if success_history:
                axes[1].plot(success_history, label='Success')
                axes[1].plot(avg_success_history, label='Average Success')
                axes[1].set_ylim(0, 1.05)
            axes[1].set_title('Episode Success (info[is_success])')
            axes[1].set_xlabel('Episode')
            axes[1].set_ylabel('Success Flag')
            axes[1].legend()
            axes[1].grid(True, linestyle='--', alpha=0.5)

            plt.tight_layout()
            display(Markdown(f"### Run {run_idx} Learning Results"))
            display(fig)
            plt.close(fig)

学習済みGAILポリシーを実行し、GIFを保存して得られた報酬を確認します。

In [ ]:
gif_base = '../Results/GAIL/GAIL Policy'
gif_path = f"{gif_base}.gif"

gail_reward, success = agent.test_model(env=env, steps=200, render_save_path=gif_base, fps=5)
success_str = "成功" if success else "失敗"
print(f'GAILの報酬: {gail_reward:.3f} - タスク: {success_str}')

if Path(gif_path).exists():
    display(Markdown('### GAILポリシーの実行結果 (GIF)'))
    display(Image(filename=gif_path))
else:
    print(f"GIF が見つかりませんでした: {gif_path}")

X connection to :0 broken (explicit kill or server shutdown).


: 